In [18]:
import polars as pl

train = pl.read_csv("/home/shinichiro.saito/atmacup17/data/train_with_index.csv")

In [19]:
train

Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count
i64,i64,str,str,i64,i64,i64
0,25,"""3-season skirt!""","""Adorable, well-made skirt! lin…",5,1,4
0,39,"""Very cute""","""Love the asymmetrical hem. wai…",5,1,0
0,42,"""Beautiful! fruns small for typ…","""I love this skirt! i wasn't su…",5,1,5
0,45,null,"""I was really pleased with this…",5,1,9
0,57,"""Unique, pretty asymmetric skir…","""I saw this skirt in retailer s…",5,1,1
…,…,…,…,…,…,…
232,57,"""Runs big on top""",null,3,1,5
232,58,null,"""I loved the dress, but just no…",1,1,5
232,60,"""I was really disappointed""","""I was really hoping this dress…",2,0,7


In [20]:
train = train.with_columns(
    pl.concat_str(pl.col("Rating"), pl.col("Recommended IND"), separator="_").alias(
        "rate_rec"
    )
)

In [23]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

fold_arr = np.zeros(train.height)
sgkf = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)

for idx, (_, val_idx) in enumerate(sgkf.split(train, train["rate_rec"])):
    fold_arr[val_idx] = idx

train = train.with_columns(pl.Series(fold_arr).cast(pl.Int64).alias("fold"))

In [25]:
train.filter(pl.col("fold") == 0)["rate_rec"].value_counts().sort("rate_rec")

rate_rec,count
str,u32
"""1_0""",117
"""1_1""",4
"""2_0""",211
"""2_1""",12
"""3_0""",244
"""3_1""",170
"""4_0""",24
"""4_1""",701
"""5_0""",4


In [27]:
# idとlabelをdictにして保存する
id_fold_dict = dict(zip(train["Clothing ID"], train["fold"]))

In [28]:
# jsonで保存する
import json

with open(
    f"/home/shinichiro.saito/atmacup17/data/label_stratified_fold.json", "w"
) as f:
    json.dump(id_fold_dict, f)